##### 生成线性方程组的哈密顿量，使用经典的方法(矩阵特征值分解)求解，看是否与符合理论推导

In [1]:
# 导入必要的库
import matplotlib.pyplot as plt

import pennylane as qml
import numpy as np
from hamiltonian_until import linear_equation_hamiltonian, binary_conversion


使用事先实现的函数 linear_equation_hamiltonian 即可得到缩放后的线性方程组的哈密顿量

In [2]:
# 定义矩阵 A 和向量 b
A_matrix = np.array([[2, 5, -13],
              [1, -3, 1],
              [-5, 6, 8]]).astype(float)

b_vec = np.array([1000, 0, -600]).astype(float)

x_vec = np.array([1200, 500, 300]).astype(float)

# 对向量 b 和 x 进行缩放，以方便使用 4 位的二进制进行表示 x 中的每个元素
# 分别为 1100 0101 0011
b_vec = b_vec/100
x_vec = x_vec/100

# 每个变量用 num_qubits 进行编码，总共需要 tol_qubits
num_qubits = 4
tol_qubits = 12

# 生成对应的线性方程组哈密顿量
coeffs, obs  = linear_equation_hamiltonian(A_matrix, b_vec, num_qubits)
hamiltonian = qml.Hamiltonian(coeffs, obs)

print(coeffs)
print(obs)
print(hamiltonian)

[2.0, -2.0, 8.0, -8.0, 32.0, -32.0, 128.0, -128.0, 12.5, -12.5, 50.0, -50.0, 200.0, -200.0, 800.0, -800.0, 84.5, -84.5, 338.0, -338.0, 1352.0, -1352.0, 5408.0, -5408.0, 0.5, -0.5, 2.0, -2.0, 8.0, -8.0, 32.0, -32.0, 4.5, -4.5, 18.0, -18.0, 72.0, -72.0, 288.0, -288.0, 0.5, -0.5, 2.0, -2.0, 8.0, -8.0, 32.0, -32.0, 12.5, -12.5, 50.0, -50.0, 200.0, -200.0, 800.0, -800.0, 18.0, -18.0, 72.0, -72.0, 288.0, -288.0, 1152.0, -1152.0, 32.0, -32.0, 128.0, -128.0, 512.0, -512.0, 2048.0, -2048.0, 2.0, -2.0, -2.0, 2.0, 4.0, -4.0, -4.0, 4.0, 8.0, -8.0, -8.0, 8.0, 2.0, -2.0, -2.0, 2.0, 8.0, -8.0, -8.0, 8.0, 16.0, -16.0, -16.0, 16.0, 4.0, -4.0, -4.0, 4.0, 8.0, -8.0, -8.0, 8.0, 32.0, -32.0, -32.0, 32.0, 8.0, -8.0, -8.0, 8.0, 16.0, -16.0, -16.0, 16.0, 32.0, -32.0, -32.0, 32.0, 12.5, -12.5, -12.5, 12.5, 25.0, -25.0, -25.0, 25.0, 50.0, -50.0, -50.0, 50.0, 12.5, -12.5, -12.5, 12.5, 50.0, -50.0, -50.0, 50.0, 100.0, -100.0, -100.0, 100.0, 25.0, -25.0, -25.0, 25.0, 50.0, -50.0, -50.0, 50.0, 200.0, -200.0, -200.0

将 pennylane 哈密顿量的格式转换成矩阵形式，即可使用特征值分解获得该哈密顿量的基态及其基态能量

In [5]:
# 哈密顿量转换成矩阵，并计算最小特征值及其特征向量
hamiltonian_sparse = hamiltonian.sparse_matrix()
hamiltonian_matrix = hamiltonian_sparse.toarray()

# 舍弃矩阵 hamiltonian_matrix 的虚部，仅保留实部，本身哈密顿量就是没有虚部
hamiltonian_matrix_real = hamiltonian_matrix.real

# 计算特征值和特征向量
eigenvalues, eigenvectors = np.linalg.eig(hamiltonian_matrix_real.astype(float))

# eigenvalues = eigenvalues.astype(float)

# 找到最小特征值的索引
min_index = np.argmin(eigenvalues.astype(float))
# 最小特征值
min_eigenvalue = eigenvalues[min_index]
# 对应的特征向量
min_eigenvector = eigenvectors[:, min_index]

print("min_eigenvector = ", min_eigenvector)
print("bin = ", binary_conversion(min_eigenvector))
print("min_index = ", min_index)
print("min_eigenvalue = ", min_eigenvalue)
print("eigenvalues = ", eigenvalues)

min_eigenvector =  [0. 0. 0. ... 0. 0. 0.]
bin =  (['001110101100'], [1.0])
min_index =  940
min_eigenvalue =  -136.0
eigenvalues =  [    0. 17824.  5168. ... 14410.  6338. 29970.]


可以看到，哈密顿量对角元上序号为 940 的数值，即是该哈密顿量的基态能量，对应的单位向量即为基态。将其转换成态矢量的写法，应为 |0011 1010 1100>，其中 0011、1010、1100 分别代表解 x1、x2、x3，例如将 0011 代入到推导中的二进制公式有 0×1+0×2+1×4+1×8 = 12，再乘上缩放系数 100 等于 1200，这正是该问题的正确答案。也可以理解为，每隔 4 个 qubits，我们需要将答案的顺序逆序(因为在计算机中，从右到左第0位是1，第1位是2，以此类推)，即 0011 逆序后为 1100，再使用二进制转十进制函数，也可获得正确答案的缩放。

In [4]:
# 答案应为 0011 1010 1100
# 需要对答案进行倒转(每 4 qubit)，即 1100 0101 0011，而后再转换成十进制即可得到正确答案
# 这是因为编码的时候，顺序便是倒转过来的
print(int('1100', 2))
print(int('0101', 2))
print(int('0011', 2))

12
5
3
